In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas import json_normalize
import gc

from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import xgboost as xgb

In [2]:
today = datetime.today().strftime('%Y%m%d')
year_list =[2020, 2021, 2022]
df_ho =pd.DataFrame([])
for i in year_list:
    today_year = i

    key = 'UhnjyvmWEq5SH4yCqPylCPQSp4NdA524up%2FM1CrsUf3143Vmj3Rwx7PGHxq5YZY4mFHNHQ%2F0ixGwapRFApPsaw%3D%3D'
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear=' + str(today_year) + '&ServiceKey=' + str(key)
    response = requests.get(url)
    if response.status_code == 200:
        json_ob = json.loads(response.text)
        holidays_data = json_ob['response']['body']['items']['item']
        dataframe = json_normalize(holidays_data)
    df_ho = pd.concat([df_ho, dataframe])
df_ho=df_ho.reset_index()
df_ho['locdate']=df_ho['locdate'].reset_index(drop=True)

In [3]:
def csv_to_parquet(csv_path, save_name):
    df = pd.read_csv(csv_path)
    df.to_parquet(f'./{save_name}.parquet')
    del df
    gc.collect()
    print(save_name, 'Done.')

In [4]:
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/train.csv', 'train')
csv_to_parquet('C:/Users/82108/JejuRoad/JejuRoadData/open/test.csv', 'test')

train Done.
test Done.


In [5]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')

In [6]:
str_col = ['start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [7]:
y = train['target'] 

x = train.drop(['id','target' , 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code',"start_node_name",'start_latitude','start_longitude','end_latitude','end_longitude'], axis=1)

test = test.drop(['id' , 'end_node_name','vehicle_restricted','road_type','weight_restricted','height_restricted','multi_linked','road_rating','connect_code',"start_node_name",'start_latitude','start_longitude','end_latitude','end_longitude'], axis=1)

print(x.shape)
print(y.shape)
print(test.shape)

(4701217, 8)
(4701217,)
(291241, 8)


In [8]:
for i in (df_ho['locdate']):
    x.loc[(x['base_date'] == i),'base_date'] = 1
x.loc[(x['base_date'] > 1),'base_date'] = 0

for i in (df_ho['locdate']):
    test.loc[(test['base_date'] == i),'base_date'] = 1
    
test.loc[(test['base_date'] > 1),'base_date'] = 0

In [9]:
col_list = ['base_hour','lane_count','day_of_week','maximum_speed_limit','road_name']
for i in col_list:
    x = pd.get_dummies(x,columns = [i])
    test = pd.get_dummies(test,columns = [i])

In [10]:
temp1=[]
for i in (x.columns):
    temp1.append(i)
    print('"{}"'.format(i),end=",")

"base_date","start_turn_restricted","end_turn_restricted","base_hour_0","base_hour_1","base_hour_2","base_hour_3","base_hour_4","base_hour_5","base_hour_6","base_hour_7","base_hour_8","base_hour_9","base_hour_10","base_hour_11","base_hour_12","base_hour_13","base_hour_14","base_hour_15","base_hour_16","base_hour_17","base_hour_18","base_hour_19","base_hour_20","base_hour_21","base_hour_22","base_hour_23","lane_count_1","lane_count_2","lane_count_3","day_of_week_금","day_of_week_목","day_of_week_수","day_of_week_월","day_of_week_일","day_of_week_토","day_of_week_화","maximum_speed_limit_30.0","maximum_speed_limit_40.0","maximum_speed_limit_50.0","maximum_speed_limit_60.0","maximum_speed_limit_70.0","maximum_speed_limit_80.0","road_name_-","road_name_경찰로","road_name_고평교","road_name_관광단지1로","road_name_관광단지2로","road_name_관광단지로","road_name_관덕로","road_name_권학로","road_name_남조로","road_name_동문로","road_name_동부관광도로","road_name_동홍로","road_name_번영로","road_name_산서로","road_name_삼무로","road_name_삼봉로","road_na

In [11]:
temp2=[]
for i in (test.columns):
    temp2.append(i)
    #print('"{}"'.format(i),end=",")

In [12]:
s = set(temp2)
temp3 = [x for x in temp1 if x not in s]

In [13]:
for i in temp3:
    test["{}".format(i)] = 0

In [14]:
test = test[["base_date","start_turn_restricted","end_turn_restricted","base_hour_0","base_hour_1","base_hour_2","base_hour_3","base_hour_4","base_hour_5","base_hour_6","base_hour_7","base_hour_8","base_hour_9","base_hour_10","base_hour_11","base_hour_12","base_hour_13","base_hour_14","base_hour_15","base_hour_16","base_hour_17","base_hour_18","base_hour_19","base_hour_20","base_hour_21","base_hour_22","base_hour_23","lane_count_1","lane_count_2","lane_count_3","day_of_week_금","day_of_week_목","day_of_week_수","day_of_week_월","day_of_week_일","day_of_week_토","day_of_week_화","maximum_speed_limit_30.0","maximum_speed_limit_40.0","maximum_speed_limit_50.0","maximum_speed_limit_60.0","maximum_speed_limit_70.0","maximum_speed_limit_80.0","road_name_-","road_name_경찰로","road_name_고평교","road_name_관광단지1로","road_name_관광단지2로","road_name_관광단지로","road_name_관덕로","road_name_권학로","road_name_남조로","road_name_동문로","road_name_동부관광도로","road_name_동홍로","road_name_번영로","road_name_산서로","road_name_삼무로","road_name_삼봉로","road_name_삼성로","road_name_새서귀로","road_name_서사로","road_name_수영장길","road_name_시민광장로","road_name_시청로","road_name_신광로","road_name_신대로","road_name_신산로","road_name_아봉로","road_name_애원로","road_name_애조로","road_name_어시천교","road_name_연동로","road_name_연북2교","road_name_연북로","road_name_연삼로","road_name_외도천교","road_name_일반국도11호선","road_name_일반국도12호선","road_name_일반국도16호선","road_name_일반국도95호선","road_name_일반국도99호선","road_name_일주동로","road_name_임항로","road_name_제2거로교","road_name_중문로","road_name_중산간서로","road_name_중앙로","road_name_중정로","road_name_지방도1112호선","road_name_지방도1115호선","road_name_지방도1116호선","road_name_지방도1117호선","road_name_지방도1118호선","road_name_지방도1119호선","road_name_지방도1120호선","road_name_지방도1132호선","road_name_지방도1136호선","road_name_지방도97호선","road_name_첨단로","road_name_태평로","road_name_한천로","road_name_호근로","road_name_호서중앙로"]]

In [15]:
x = x.to_numpy()
y = y.to_numpy()
test = test.to_numpy()

In [16]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [26]:
del x, y

##### import matplotlib.pyplot as plt
alhpa_list = [3,4,5,6,7,8,9]
train_score = []
test_score = []
for i in tqdm(alhpa_list):
    xg_reg = xgb.XGBRegressor(max_depth=i,random_state=42).fit(x_train, y_train)
    train_score.append(xg_reg.score(x_train, y_train))
    test_score.append(xg_reg.score(x_test, y_test))

##### plt.plot(np.log10(alhpa_list), train_score)
plt.plot(np.log10(alhpa_list), test_score)
plt.xlabel("alpha")
plt.ylabel("max_depth")
plt.show()

In [46]:
xg_reg = xgb.XGBRegressor(max_depth=10,random_state=42).fit(x_train, y_train)

In [ ]:
xg_reg = xgb.XGBRegressor(max_depth=12,random_state=42).fit(x, y)

In [47]:
print(xg_reg.score(x_train, y_train))
print(xg_reg.score(x_test, y_test))

0.7035138433716754
0.6987258841880919


In [ ]:
pred = xg_reg.predict(test)
sample_submission = pd.read_csv('C:/Users/82108/JejuRoad/JejuRoadData/open/sample_submission.csv')
sample_submission['target'] = pred
sample_submission.to_csv("C:/Users/82108/JejuRoad/JejuRoadData/open/submit.csv", index = False)

In [ ]:
sample_submission

In [34]:
sample_submission

,id,target
0,TEST_000000,25.270121
1,TEST_000001,46.050335
2,TEST_000002,56.191345
3,TEST_000003,30.591667
4,TEST_000004,40.821388
...,...,...
291236,TEST_291236,50.665264
291237,TEST_291237,44.236691
291238,TEST_291238,23.385305
291239,TEST_291239,25.674559
